In [1]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
#import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.lstur import LSTURModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 16:07:37) 
[GCC 9.3.0]
Tensorflow version: 2.9.1


# LSTUR: Neural News Recommendation with Long- and Short-term User Representations

https://github.com/microsoft/recommenders/blob/main/examples/00_quick_start/lstur_MIND.ipynb

In [2]:
epochs = 5
seed = 40
batch_size = 32

# Options: demo, small, large
MIND_type = 'large'

In [3]:
import inspect
lines = inspect.getsource(LSTURModel)
print(lines)

class LSTURModel(BaseModel):
    """LSTUR model(Neural News Recommendation with Multi-Head Self-Attention)

    Mingxiao An, Fangzhao Wu, Chuhan Wu, Kun Zhang, Zheng Liu and Xing Xie:
    Neural News Recommendation with Long- and Short-term User Representations, ACL 2019

    Attributes:
        word2vec_embedding (numpy.ndarray): Pretrained word embedding matrix.
        hparam (object): Global hyper-parameters.
    """

    def __init__(self, hparams, iterator_creator, seed=None):
        """Initialization steps for LSTUR.
        Compared with the BaseModel, LSTUR need word embedding.
        After creating word embedding matrix, BaseModel's __init__ method will be called.

        Args:
            hparams (object): Global hyper-parameters. Some key setttings such as type and gru_unit are there.
            iterator_creator_train (object): LSTUR data loader class for train data.
            iterator_creator_test (object): LSTUR data loader class for test and validation data
       

### Dowload data

In [2]:
cwd = os.getcwd()
cwd

'/ssd003/home/bjimenez/news_recommender'

In [3]:
cwd = os.getcwd()
data_path = cwd + '/data/' #tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'dev', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'dev', r'behaviors.tsv')

data_path_LSTUR = cwd + '/data_LSTUR/' 
wordEmb_file = os.path.join(data_path_LSTUR, "utils", "embedding.npy")
userDict_file = os.path.join(data_path_LSTUR, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path_LSTUR, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path_LSTUR, "utils", r'lstur.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
    
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path_LSTUR, 'utils'), mind_utils)

In [4]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 32, 'show_step': 100000, 'title_size': 30, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'lstur', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/ssd003/home/bjimenez/news_recommender/data_LSTUR/utils/embedding.npy', 'wordDict_file': '/ssd003/home/bjimenez/news_recommender/data_LSTUR/utils/word_dict.pkl', 'userDict_file': '/ssd003/home/bjimenez/news_recommender/data_LSTUR/utils/uid2index.pkl'}


In [5]:
iterator = MINDIterator

### Train the LSTUR model


In [6]:
model = LSTURModel(hparams, iterator, seed=seed)

Tensor("conv1d/Relu:0", shape=(None, 30, 400), dtype=float32)
Tensor("att_layer2/Sum_1:0", shape=(None, 400), dtype=float32)


/ssd003/projects/aieng/public/recsys/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
valid_news_file

'/ssd003/home/bjimenez/news_recommender/data/dev/news.tsv'

In [8]:
print(model.run_eval(valid_news_file, valid_behaviors_file))

0it [00:00, ?it/s]/ssd003/projects/aieng/public/recsys/lib/python3.7/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-11-28 10:43:48.494221: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-28 10:43:48.496181: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2251it [00:08, 270.35it/s]
11765it [07:16, 26.98it/s]
376471it [01:02, 5999.20it/s]


{'group_auc': 0.5248, 'mean_mrr': 0.2256, 'ndcg@5': 0.2341, 'ndcg@10': 0.2977}


In [ ]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

0it [00:00, ?it/s]2022-11-28 11:05:22.551121: W tensorflow/c/c_api.cc:349] Operation '{name:'user_encoder/gru/while' id:1585 op device:{requested: '', assigned: ''} def:{{{node user_encoder/gru/while}} = While[T=[DT_INT32, DT_INT32, DT_INT32, DT_VARIANT, DT_FLOAT, 3049133034371594848, DT_VARIANT, DT_VARIANT, DT_VARIANT, DT_VARIANT, DT_VARIANT], _lower_using_switch_merge=true, _num_original_outputs=113, _read_only_resource_inputs=[9, 10, 11], _stateful_parallelism=false, body=user_encoder_gru_while_body_2198[], cond=user_encoder_gru_while_cond_2197[], output_shapes=[[], [], [], [], [?,400], 5512068267592963614, [], [], [], [], []], parallel_iterations=32](user_encoder/gru/while/loop_counter, user_encoder/gru/while/maximum_iterations, user_encoder/gru/time, user_encoder/gru/TensorArrayV2_1, user_encoder/gru/zeros_like, user_encoder/reshape_1/Reshape, user_encoder/gru/strided_slice, user_encoder/gru/TensorArrayUnstack/TensorListFromTensor, user_encoder/gru/TensorArrayUnstack_1/TensorListF

at epoch 1
train info: logloss loss:1.2636026011447679
eval info: group_auc:0.6819, mean_mrr:0.329, ndcg@10:0.4268, ndcg@5:0.3628
at epoch 1 , train time: 24458.9 eval time: 846.3


45628it [2:54:52,  4.30it/s]

In [ ]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

In [ ]:
#sb.glue("res_syn", res_syn)

### Save the model

In [ ]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "lstur_ckpt"))

### Output prediction file

In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

In [ ]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

In [ ]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()


Reference

[1] Mingxiao An, Fangzhao Wu, Chuhan Wu, Kun Zhang, Zheng Liu and Xing Xie: Neural News Recommendation with Long- and Short-term User Representations, ACL 2019

[2] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html

[3] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/
